# ZNEUS PROJECT - DIABETES 130-US

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. EXPLORATORY DATA ANALYSIS (EDA)

This dataset represents ten years woth of clinical care data at 130 US hospitals and integrated delivery networks. The instances represent hospitalized patient records diagnosed with diabetes, who underwent laboratory, medications, and stayed up to 14 days. The main question we will be focusing on is the impact of medication on the stay and readmission of patients.

source: https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008

In [2]:
df = pd.read_csv("diabetic_data.csv")
pd.set_option("display.max_columns", 60)
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


As we can see from the display, we are facing a multitude of problems. For one, the ages and weights have been kept anonymous and have been replaced with a range and question mark sign respectively. There is also redundant data, such as the patient id and encounter id. We will need to get rid of this redundant data.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In order for us to clean up the data, we'll need a description of what exactly we are looking at. In the following paragraph, we will be explaining, what each of the columns mean. This will help us with selecting relevant columns for training.

- _encounter_id_: Unique index used to differentiate cases.
- _patient_nbr_, _race_, _gender_, _age_, _weight_: Personal information for each specific patient (name should be explainable).
- _admission_type_id_, _admission_source_id_: Circumstances at which the patient was admitted in.
- _discharge_disposition_, _time_in_hospital_: Circumstances at which the patient was discharged and time of stay.
- _payer_code_: Code of the person, who payed for the procedures.
- _medical_specialty_: Speciality of the physician in charge of the patient.
- _num_lab_procedures_, num_procedures : Number of laboratory and non-laboratory procedures done during the patients stay.
- _num_medications_: Number of generic names administered during the procedures.
- _number_outpatient_, _number_emergency_, _number_inpatient_: Number of visits in the last year preceeding the admission.
- _diag_1-3_, _number_diagnoses_: Number of diagnosses and the primary/secondary/additional secondary diagnosesses.
- _max_glu_serum_: Indicates the result of the glu_serum test, alternatively, wheter it was taken.
- _A1Cresult_: Indicates the result of the A1C test, alternatively, wheter it was taken.
- _metformin_ - _metformin-pioglitazone_: Indicates wheter the dosage of the given drug has increaased, decreased, stayed same or wasn't prescribed at all.
- _change_, _diabetesMed_: Indicates change in the diabetic medication and if any drugs were prescribed.
- _readmitted_: Indicates if and when a patient had been readmitted after treatment.

Using info(), we can see that the dataset contains 50 columns in total, which is good, because we can train the network more efficiently this way. 

Most of the data are either obejcts or ints, which is perfect for us. From the previous viewing, we can see, that the object data is mostly of the Up/Down/Steady/No variety. We'll be transforming these columns to mathematical constants (e.g. Up = 1, Down = 0, Steady = 0.5, No = -1)

We can notice, that the _max_glu_serum_ and _A1Cresult_ columns contain a lot of null-values. While the other values do not display null-values using this method, we can still clearly see from the display, that they hide this issue by inserting "?" instead of null-values. I understand this is to protect sensitive data but it's really annoying for machine learning proccesses.